***In this notebook, we will ignore the instances of the majority class to catch rules from the minority classes.***

***1) Infants born to mothers with pre-existing health conditions such as diabetes, hypertension, or hepatitis may be at increased risk of specific health problems, including low birth weight, preterm birth, and congenital anomalies.***

In [1]:
import pandas as pd 
import random
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import rdflib
from rdflib import Graph, Namespace, URIRef, OWL, Literal
from rdflib.namespace import RDF, RDFS, OWL, XSD
from rdflib import RDF, RDFS, OWL, Namespace, Graph
import re
from tqdm import tqdm
from sklearn.utils import resample
import csv
import rdflib
from rdflib import Graph
#from apyori import apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from itertools import groupby

In [2]:
# Define the chunk size
chunk_size = 100000

# specify the file path
file_path = 'US_birth.csv'

# Initialize an empty list to store chunks
chunks = []

# read the file in chunks
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    chunks.append(chunk)

# Concatenate the chunks to a single dataframe
df = pd.concat(chunks, ignore_index=True)
df= df.drop('Unnamed: 0',axis= 1)
df.head()

C:\Users\mouad\AppData\Local\Temp\ipykernel_19188\3464066820.py:11: DtypeWarning: Columns (59,68,92,93,94,99) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\mouad\AppData\Local\Temp\ipykernel_19188\3464066820.py:11: DtypeWarning: Columns (19,20,21,53,68,79,99,112) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\mouad\AppData\Local\Temp\ipykernel_19188\3464066820.py:11: DtypeWarning: Columns (53,94) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\mouad\AppData\Local\Temp\ipykernel_19188\3464066820.py:11: DtypeWarning: Columns (94) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\mouad\AppData\Local\Temp\ipyker

,BirthMonth,TimeofBirth,BirthDayofWeek,BirthPlace,MotherSingleYearofAge,MotherAge,MotherNativity,ResidenceStatus,MotherRaceRecode31,MotherRaceRecode6,...,CleftLipw/orw/oCleftPalate,CleftPalatealone,DownSyndrome,SuspectedChromosomalDisorder,Hypospadias,NoCongenitalAnomaliesChecked,InfantTransferred,InfantLivingatTimeofReport,InfantBreastfedatDischarge,SmokeStatus
0,April,2329,Tuesday,Hospital,33,30-34 Years,Born in the U.S. (50 US States),RESIDENT,White (only),White (only),...,No,No,No,No,No,True,No,Yes,Yes,Nonsmoker
1,May,455,Friday,Hospital,28,25-29 Years,Born in the U.S. (50 US States),RESIDENT,White (only),White (only),...,No,No,No,No,No,True,No,Yes,Yes,Nonsmoker
2,March,1357,Wednesday,Hospital,23,20-24 Years,Born in the U.S. (50 US States),RESIDENT,Black (only),Black (only),...,No,No,No,No,No,True,No,Yes,Yes,Nonsmoker
3,April,1506,Wednesday,Hospital,22,20-24 Years,Born in the U.S. (50 US States),INTRASTATE NONRESIDENT,White (only),White (only),...,No,No,No,No,No,True,No,Yes,Yes,Nonsmoker
4,October,2319,Wednesday,Hospital,19,15-19 Years,Born in the U.S. (50 US States),INTRASTATE NONRESIDENT,White (only),White (only),...,No,No,No,No,No,True,No,Yes,Yes,Smoker


In [3]:
for col in df.columns:
    print(col)

BirthMonth
TimeofBirth
BirthDayofWeek
BirthPlace
MotherSingleYearofAge
MotherAge
MotherNativity
ResidenceStatus
MotherRaceRecode31
MotherRaceRecode6
MotherHispanicOriginRecode
PaternityAcknowledged
MaritalStatus
MotherEducation
FatherAge
FatherRaceRecode6
FatherHispanicOriginRecode
FatherEducation
PriorBirthsNowLiving
PriorBirthsNowDead
PriorOtherTerminations
LiveBirthOrderRecode
TotalBirthOrderRecode
IntervalSinceLastLiveBirthRecode11
IntervalSinceLastOtherPregnancyRecode11
IntervalSinceLastPregnancyRecode11
MonthPrenatalCareBeganRecode
NumberofPrenatalVisits
NumberofPrenatalVisitsRecode
WIC
CigarettesBeforePregnancyRecode
Cigarettes1stTrimesterRecode
Cigarettes2ndTrimesterRecode
Cigarettes3rdTrimesterRecode
MotherHeightinTotalInches
BodyMassIndex
BodyMassIndexRecode
Pre-pregnancyWeightRecode
DeliveryWeightRecode
WeightGain
WeightGainRecode
Pre-pregnancyDiabetes
GestationalDiabetes
Pre-pregnancyHypertension
GestationalHypertension
HypertensionEclampsia
PreviousPretermBirth
Infertility

In [4]:
selected_columns = ['GestationalDiabetes','GestationalHypertension','HepatitisB','HepatitisC','HypertensionEclampsia','Gastroschisis','BirthWeightDetailInGrams','Anencephaly','Antibiotics','Antibiotics','AsstReproductiveTechnology','Chlamydia','Chorioamnionitis','CongenitalDiaphragmaticHernia','CyanoticCongenitalHeartDisease','Gastroschisis']

In [5]:
#select columns
selected_df = df.loc[:, [col.lower() in map(str.lower, selected_columns) for col in df.columns]]
selected_df.shape

(999993, 14)

In [6]:
for col in selected_df.columns:
    print(selected_df[col].value_counts(),end='\n \n')

No                       923845
Yes                       75176
Unknown or not stated       972
Name: GestationalDiabetes, dtype: int64
 
No                       920032
Yes                       78989
Unknown or not stated       972
Name: GestationalHypertension, dtype: int64
 
No                       996617
Yes                        2404
Unknown or not stated       972
Name: HypertensionEclampsia, dtype: int64
 
Not applicable           981793
Yes                       12659
No                         4200
Unknown or not stated      1341
Name: AsstReproductiveTechnology, dtype: int64
 
No                       981343
Yes                       15773
Unknown or not stated      2877
Name: Chlamydia, dtype: int64
 
No                       995031
Unknown or not stated      2877
Yes                        2085
Name: HepatitisB, dtype: int64
 
No                       994161
Yes                        2955
Unknown or not stated      2877
Name: HepatitisC, dtype: int64
 
No               

In [7]:
col = 'GestationalDiabetes'
# Identify the majority class
majority_class = selected_df[col].value_counts().idxmax()

# Get the indices of instances in the majority class
majority_indices = selected_df[selected_df[col] == majority_class].index

# Remove the majority class
selected_df.drop(majority_indices, inplace=True)

C:\Users\mouad\AppData\Local\Temp\ipykernel_19188\288259970.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df.drop(majority_indices, inplace=True)


In [8]:
selected_df.shape

(76148, 14)

In [9]:
# create a new RDF graph
graph = Graph()

# create a namespace for your ontology
ont_namespace = Namespace('http://www.semanticweb.org/mypc/ontologies/2022/11/untitled-ontology-22#')

# add your ontology to the graph
graph.parse('C:/Users/mouad/Desktop/upwork/US Birth/data/BirthOntology.owl', format='xml')

<Graph identifier=N7a1e049b4ff841f2b193f45d9d074ab7 (<class 'rdflib.graph.Graph'>)>

In [10]:
# Get all object properties
object_properties = list(graph.subjects(RDF.type, OWL.ObjectProperty))
clean_object_properties =  [str(object_propertiy).split('has')[1] for object_propertiy in object_properties]
# Get all data properties
data_properties = list(graph.subjects(RDF.type, OWL.DatatypeProperty))
clean_data_properties = [str(data_property).split('#')[1] for data_property in data_properties]

In [11]:
# iterate through the rows in the DataFrame
for index, row in tqdm(selected_df.iterrows()):
    # generate a URI for the subject
    subject_uri = ont_namespace[f"BR{index}"]
    
    # iterate through the columns in the row
    for col_name, col_value in row.items():
        if col_name in clean_object_properties:
            # create a URIRef for the property
            property_uri = ont_namespace['has'+str(col_name)]
            # generate a URI for the literal value
            literal_uri = ont_namespace[f"{col_value.replace(' ', '')}"]
            # create a Literal object for the value with datatype URI
            literal = Literal(col_value, datatype=literal_uri)
        elif col_name in clean_data_properties: 
            # create a URIRef for the property
            property_uri = ont_namespace[col_name]
            # create a Literal object for the value
            literal = Literal(col_value)
        
        # add the triple to the graph
        graph.add((subject_uri, property_uri, literal))

76148it [00:33, 2297.26it/s]


In [12]:
# Get all the unique subjects in the graph
subjects = set(s for s, _, _ in graph)

In [13]:
len(subjects)

76943

In [14]:
# Sort the triples by subject
sorted_triples = sorted(graph, key=lambda triple: triple[0])
transactions = []
# Group the triples by subject
for subject, triples in tqdm(groupby(sorted_triples, key=lambda triple: triple[0])):
    # Convert the triples for the current subject to a list of strings
    triples_str = [f"{p.split('#')[-1]} : {str(o) if not isinstance(o, rdflib.term.URIRef) else o.split('#')[-1]}" 
                   for _, p, o in triples]
    # Append the list of triples for the current subject to the transactions list
    transactions.append(triples_str)

76943it [00:01, 67019.59it/s]


In [15]:
# Encode the transaction data
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
encoded_df = pd.DataFrame(te_ary, columns=te.columns_)
encoded_df.head()

,Age : 17,Age : 19,Age : 20,Age : 21,Age : 22,Age : 23,Age : 24,Age : 25,Age : 26,Age : 27,...,type : Seizures,type : SmokeStatus,type : Steroids,type : SuccessfulExternalCephalicVersion,type : Surfactant,type : SuspectedChromosomalDisorder,type : Syphilis,type : TrialOfLaborAttemptedIfCesarean,type : UnplannedHysterectomy,type : Variable
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [16]:
# Generate frequent itemsets using Apriori algorithm
frequent_itemsets = apriori(encoded_df, min_support=0.05, use_colnames=True)
#gest_diabetes_itemsets = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: 'hasGestationalDiabetes : Yes' in x)]

In [17]:
df_results = pd.DataFrame(frequent_itemsets)
df_results.head()

,support,itemsets
0,0.988849,(hasAnencephaly : No)
1,0.677995,(hasAntibiotics : No)
2,0.313180,(hasAntibiotics : Yes)
3,0.949092,(hasAsstReproductiveTechnology : Not applicable)
4,0.973981,(hasChlamydia : No)


In [18]:
# Generate association rules
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.5)
# Filter rules to only include those with antecedent 'GestationalDiabetes : Yes'
filtered_rules = rules[rules['antecedents'].astype(str).str.startswith('frozenset({\'hasGestationalDiabetes : Yes\'})')]

In [19]:
df_results2 = pd.DataFrame(filtered_rules)
df_results2 = df_results2.sort_values(by='lift', ascending=False)
df_results2.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
915381,(hasGestationalDiabetes : Yes),"(hasHepatitisC : No, hasChlamydia : No, hasAss...",0.977568,0.636523,0.636523,0.651129,1.022947,0.014279,1.041867
618148,(hasGestationalDiabetes : Yes),"(hasHepatitisC : No, hasAsstReproductiveTechno...",0.977568,0.637615,0.637615,0.652246,1.022947,0.014303,1.042074
614210,(hasGestationalDiabetes : Yes),(hasAsstReproductiveTechnology : Not applicabl...,0.977568,0.556776,0.556776,0.569552,1.022947,0.012490,1.029681
1073429,(hasGestationalDiabetes : Yes),"(hasHepatitisC : No, hasAsstReproductiveTechno...",0.977568,0.806545,0.806545,0.825053,1.022947,0.018093,1.105790
1722742,(hasGestationalDiabetes : Yes),"(hasHepatitisC : No, hasChlamydia : No, hasAss...",0.977568,0.550888,0.550888,0.563530,1.022947,0.012358,1.028962
617004,(hasGestationalDiabetes : Yes),(hasAsstReproductiveTechnology : Not applicabl...,0.977568,0.556633,0.556633,0.569406,1.022947,0.012486,1.029664
928970,(hasGestationalDiabetes : Yes),"(hasHepatitisC : No, hasAsstReproductiveTechno...",0.977568,0.637602,0.637602,0.652233,1.022947,0.014303,1.042071
1727340,(hasGestationalDiabetes : Yes),"(hasChlamydia : No, hasAsstReproductiveTechnol...",0.977568,0.629388,0.629388,0.643831,1.022947,0.014119,1.040550
1074191,(hasGestationalDiabetes : Yes),"(hasHepatitisC : No, hasAsstReproductiveTechno...",0.977568,0.806545,0.806545,0.825053,1.022947,0.018093,1.105790
927826,(hasGestationalDiabetes : Yes),(hasAsstReproductiveTechnology : Not applicabl...,0.977568,0.556620,0.556620,0.569393,1.022947,0.012486,1.029662


In [20]:
df_results2.shape

(4095, 9)

In [22]:
pd.set_option('display.max_rows', None)
df_results2[['antecedents','consequents']]

,antecedents,consequents
915381,(hasGestationalDiabetes : Yes),"(hasHepatitisC : No, hasChlamydia : No, hasAss..."
618148,(hasGestationalDiabetes : Yes),"(hasHepatitisC : No, hasAsstReproductiveTechno..."
614210,(hasGestationalDiabetes : Yes),(hasAsstReproductiveTechnology : Not applicabl...
1073429,(hasGestationalDiabetes : Yes),"(hasHepatitisC : No, hasAsstReproductiveTechno..."
1722742,(hasGestationalDiabetes : Yes),"(hasHepatitisC : No, hasChlamydia : No, hasAss..."
617004,(hasGestationalDiabetes : Yes),(hasAsstReproductiveTechnology : Not applicabl...
928970,(hasGestationalDiabetes : Yes),"(hasHepatitisC : No, hasAsstReproductiveTechno..."
1727340,(hasGestationalDiabetes : Yes),"(hasChlamydia : No, hasAsstReproductiveTechnol..."
1074191,(hasGestationalDiabetes : Yes),"(hasHepatitisC : No, hasAsstReproductiveTechno..."
927826,(hasGestationalDiabetes : Yes),(hasAsstReproductiveTechnology : Not applicabl...


In [29]:
for i in range(df_results2.shape[0]):
    print(df_results2.iloc[i,1],end='\n \n \n')

frozenset({'hasHepatitisC : No', 'hasChlamydia : No', 'hasAsstReproductiveTechnology : Not applicable', 'hasAntibiotics : No', 'hasHepatitisB : No', 'hasCyanoticCongenitalHeartDisease : No', 'hasGastroschisis : No'})
 
 
frozenset({'hasHepatitisC : No', 'hasAsstReproductiveTechnology : Not applicable', 'hasChorioamnionitis : No', 'hasAntibiotics : No', 'hasHepatitisB : No', 'hasGastroschisis : No', 'hasAnencephaly : No'})
 
 
frozenset({'hasAsstReproductiveTechnology : Not applicable', 'hasChorioamnionitis : No', 'hasAntibiotics : No', 'hasCyanoticCongenitalHeartDisease : No', 'hasHypertensionEclampsia : No', 'hasGestationalHypertension : No', 'hasAnencephaly : No'})
 
 
frozenset({'hasHepatitisC : No', 'hasAsstReproductiveTechnology : Not applicable', 'hasCongenitalDiaphragmaticHernia : No', 'hasHepatitisB : No', 'hasCyanoticCongenitalHeartDisease : No', 'hasHypertensionEclampsia : No', 'hasGestationalHypertension : No'})
 
 
frozenset({'hasHepatitisC : No', 'hasChlamydia : No', 'hasA